In [29]:
#import numpy as np
import pandas as pd
import csv

#import people data from csv
import_people = pd.read_csv('../datafiles/People.csv', encoding='latin-1')
import_juniors =  pd.read_csv('../datafiles/custom/juniors.csv')
juniors = import_juniors.iloc[:, 0].values.tolist()
#juniors = import_juniors.aslist()
#import_generations = pd.read_csv('datafiles/custom/generation.csv', encoding='latin-1')

In [30]:
def jr(i, n):
    if i in juniors:
        return f"{n} Jr."
    else:
        return n

In [37]:

#remove non-players, remove unneeded columns
people_df = import_people[~import_people['playerID'].str.endswith('99')].drop(['ID', 'deathYear', 'deathMonth', 'deathDay', 'deathCountry', 'deathState', 'deathCity', 'nameGiven'], axis=1).copy()

#fix blanks and cast as integer where appropriate
people_df['birthYear'] = people_df['birthYear'].fillna(0).astype(int)
people_df['birthMonth'] = people_df['birthMonth'].fillna(0).astype(int)
people_df['birthDay'] = people_df['birthDay'].fillna(0).astype(int)
people_df['weight'] = people_df['weight'].fillna(0).astype(int)
people_df['height'] = people_df['height'].fillna(0).astype(int)
people_df['birthCountry'] = people_df['birthCountry'].fillna('Unknown').apply(lambda x: x if x != 'Ukriane' else 'Ukraine')
people_df['birthCity'] = people_df['birthCity'].fillna('Unknown')
people_df['birthState'] = people_df['birthState'].fillna('NA')
people_df['birthCity'] = people_df.apply(lambda x: f"{x['birthCity']}, {x['birthState']}", axis=1)
people_df['nameFirst'] = people_df['nameFirst'].fillna('Unknown')
people_df['bats'] = people_df['bats'].fillna('U')
people_df['throws'] = people_df['throws'].fillna('U')
people_df['bbrefID'] = people_df['bbrefID'].fillna('xxx1234')
people_df['debut'] = people_df['debut'].fillna('1800-01-01')
people_df['finalGame'] = people_df['finalGame'].fillna('1800-01-01')

#juniorize and make the full name
people_df['nameLast'] = people_df.apply(lambda x: jr(x['playerID'], x['nameLast']), axis=1)
people_df['fullName'] = people_df.apply(lambda x: f"{x['nameFirst']} {x['nameLast']}", axis=1)



print(people_df)

        playerID  birthYear  birthMonth  birthDay         birthCity  \
0      aardsda01       1981          12        27        Denver, CO   
1      aaronha01       1934           2         5        Mobile, AL   
2      aaronto01       1939           8         5        Mobile, AL   
3       aasedo01       1954           9         8        Orange, CA   
4       abadan01       1972           8        25    Palm Beach, FL   
...          ...        ...         ...       ...               ...   
20980  thomacl03       1896          11        25       Greenup, KY   
20981  malarda01       1894          10        18    White Hall, LA   
20982  gardnje02       1895           9        27  Russellville, AK   
20994   pagete01       1903           4        22       Glasgow, KY   
20995  mcdonwe01       1900           1         1       Glasgow, DE   

      birthCountry birthState nameFirst   nameLast  weight  height bats  \
0              USA         CO     David    Aardsma     215      75    R 